# Set up

In [1]:
import dash
from dash import html, dcc
from dash.dependencies import Output, Input

import geopandas as gpd

import pandas as pd

import plotly.express as px
import plotly.graph_objects as go # to create table using graphical objects

import requests


import os 
import json
from datetime import datetime

import locale

In [2]:
# #*****************************************************************************
# import plotly.io as pio
# # Ensure plots are shown in browser (during exploration)
# pio.renderers.default='browser' # set browser as renderer
# #*****************************************************************************

In [3]:
# show all outputs of cell, not merely of last line (i.e. default of Jupyter Notebook)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
# Set Belgium time (for Dutch-language indicators of last update time)
locale.setlocale(locale.LC_TIME, 'nl_BE.utf-8')

'nl_BE.utf-8'

# Fetch data

With fetching, there is apparently a limit on the amount of bikes you can fetch (i.e. " Invalid value for limit API parameter: 200 was found but -1 <= limit <= 100 is expected."). 

Therefore, we rely on the exports of the datasets and regularly update them, instead of relying on the API.

For some of the share bike brands, the exports take on a similar form / structure:
* Dott
* Bolt
* Baqme

Hence, to extract those, we can use a single function.

In [5]:
# Fetching data function (when using export function)
def fetch_data_export(url: str, brand: str):
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()  
        
        # # Convert 'last_reported' string to datetime
        # for entry in data:
        #     entry['last_reported'] = datetime.utcfromtimestamp(int(entry['last_reported'])).strftime('%Y-%m-%d %H:%M:%S')
        
        # Write new data to file
        with open(f'../data/fetched_data_sharing_bikes_{brand}.json', 'w') as json_file:
            json.dump(data, json_file) 
        print(f"Data ({brand} bikes) fetched")
        # return json_data
    
    else:
        print(f"Failed to fetch data ({brand} bikes)")

First we obtain the urls to download the json experts for the various data sets (see https://data.stad.gent/explore/?disjunctive.keyword&disjunctive.theme&sort=modified&q=deelfiets). 

In [6]:
url_dott = "https://data.stad.gent/api/explore/v2.1/catalog/datasets/dott-deelfietsen-gent/exports/json?lang=nl&timezone=Europe%2FBrussels"
# url_dott = "https://data.stad.gent//api/explore/v2.1/catalog/datasets/dott-deelfietsen-gent/records?limit=20"
# url_dott = "https://data.stad.gent/api/explore/v2.1/catalog/datasets/dott-deelfietsen-gent/records?limit=200"


url_bolt ="https://data.stad.gent/api/explore/v2.1/catalog/datasets/bolt-deelfietsen-gent/exports/json?lang=nl&timezone=Europe%2FBrussels"

url_baqme = "https://data.stad.gent/api/explore/v2.1/catalog/datasets/baqme-locaties-vrije-deelfietsen-gent/exports/json?lang=nl&timezone=Europe%2FBrussels"

In [7]:
fetch_data_export(url_dott, 'Dott')
fetch_data_export(url_bolt, 'Bolt')
fetch_data_export(url_baqme, 'Baqme')

Data (Dott bikes) fetched
Data (Bolt bikes) fetched
Data (Baqme bikes) fetched


Set up color dictionary to match brands with specific colors

In [8]:
color_dict = {
    'Dott': 'blue',
    'Bolt': 'green',
    'Baqme': 'red'
}

# Initialize data and app

Perform initial read-in of the filtered JSON file

In [9]:
with open('../data/fetched_data_sharing_bikes_Dott.json', 'r') as json_file:
    data_json_dott = json.load(json_file)

with open('../data/fetched_data_sharing_bikes_Bolt.json', 'r') as json_file:
    data_json_bolt = json.load(json_file)

with open('../data/fetched_data_sharing_bikes_Baqme.json', 'r') as json_file:
    data_json_baqme = json.load(json_file)

In [10]:
# Inspect data
data_json_dott[0].keys()
data_json_bolt[0].keys()
data_json_baqme[0].keys()

data_json_dott[0]
data_json_bolt[0]
data_json_baqme[0]

dict_keys(['bike_id', 'current_range_meters', 'is_disabled', 'is_reserved', 'last_reported', 'lat', 'lon', 'pricing_plan_id', 'rental_uris', 'vehicle_type_id', 'loc', 'time'])

dict_keys(['bike_id', 'lat', 'lon', 'current_range_meters', 'pricing_plan_id', 'vehicle_type_id', 'is_reserved', 'is_disabled', 'rental_uris', 'loc'])

dict_keys(['bike_id', 'lat', 'lon', 'is_reserved', 'is_disabled', 'vehicle_type_id', 'rental_uris', 'geopoint'])

{'bike_id': 'd34c263a-de32-4f23-ac1c-a6877e47c978',
 'current_range_meters': 56700.00000000001,
 'is_disabled': 0,
 'is_reserved': 0,
 'last_reported': '1705427263',
 'lat': 51.040778,
 'lon': 3.709855,
 'pricing_plan_id': 'd4649cd8-e995-57b6-8381-556a0b27b9cf',
 'rental_uris': '{"android": "https://go.ridedott.com/vehicles/791S5P?platform=android", "ios": "https://go.ridedott.com/vehicles/791S5P?platform=ios"}',
 'vehicle_type_id': 'dott_bicycle',
 'loc': {'lon': 3.709855, 'lat': 51.040778},
 'time': '2024-01-16T18:47:43+01:00'}

{'bike_id': 'db52e93a-d7b2-499e-ab53-7811e66b8dc8',
 'lat': 51.065098,
 'lon': 3.709198,
 'current_range_meters': 49600,
 'pricing_plan_id': '0b2ebeb0-75ec-5da0-bc35-a0f7bfdef596',
 'vehicle_type_id': '665460d8-fe6e-56e4-a3b1-2bd867304ca4',
 'is_reserved': 0,
 'is_disabled': 0,
 'rental_uris': '{"android": "https://bolt.onelink.me/sbJ2/77h7vloa?deep_link_value=bolt%253A%252F%252Faction%252FrentalsSelectVehicleByRotatedUuid%253Frotated_uuid%253Ddb52e93a-d7b2-499e-ab53-7811e66b8dc8&client_id=CITY_MAPPER", "ios": "https://bolt.onelink.me/sbJ2/77h7vloa?deep_link_value=bolt%253A%252F%252Faction%252FrentalsSelectVehicleByRotatedUuid%253Frotated_uuid%253Ddb52e93a-d7b2-499e-ab53-7811e66b8dc8&client_id=CITY_MAPPER"}',
 'loc': {'lon': 3.709198, 'lat': 51.065098}}

{'bike_id': '46a3eae8-d0d3-4347-9b5b-7067633a85ef',
 'lat': 51.046048,
 'lon': 3.742861,
 'is_reserved': 0,
 'is_disabled': 0,
 'vehicle_type_id': '4cc4d04b-3c10-480f-9196-eb6dac3e3064',
 'rental_uris': '{"android": "https://eu-mobility.joyride.tech/api/v1/gbfs/rental?bike_id=46a3eae8-d0d3-4347-9b5b-7067633a85ef&platform=android", "ios": "https://eu-mobility.joyride.tech/api/v1/gbfs/rental?bike_id=46a3eae8-d0d3-4347-9b5b-7067633a85ef&platform=ios"}',
 'geopoint': {'lon': 3.742861, 'lat': 51.046048}}

Then we add to each datapoint a key regarding their brand. The Baqme bikes also have no indication of their current range. So we add an empty key for this. Then we putt all those datapoints together so we can use them as 1 constant stream of input for the graph.  

In [11]:
def add_brand(json_list: list, brand: str):
    # color_dict = {
    #     'Dott': 'blue',
    #     'Bolt': 'green',
    #     'Baqme': 'red'
    # }
    
    for datapoint in json_list:
        datapoint['brand'] = brand
        # datapoint['color'] = color_dict[brand]

In [12]:
add_brand(data_json_dott, 'Dott')
add_brand(data_json_bolt, 'Bolt')
add_brand(data_json_baqme, 'Baqme')

# # Inspect data
# data_json_dott[0]
# data_json_bolt[0]
# data_json_baqme[0]

In [13]:
for data_point in data_json_baqme:
    data_point["current_range_meters"] = None

In [14]:
data_json_dott_bolt_baqme = data_json_dott + data_json_bolt + data_json_baqme

# Inspect data
len(data_json_dott_bolt_baqme)

880

In [15]:
# # # Convert 'last_reported' string to datetime
# # for entry in data_dott:
# #     entry['last_reported'] = datetime.utcfromtimestamp(int(entry['last_reported'])).strftime('%Y-%m-%d %H:%M:%S')

# # Transform to dataframe for easier handling
# df_dott = pd.DataFrame(data_dott_json)

In [16]:
# # Inspect dataframe
# df_dott.head()
# df_dott.tail()
# df_dott.columns
# df_dott.shape

In [17]:
# data_dott_json

In [18]:
# df_dott.columns

In [19]:
# df_dott["vehicle_type_id"].values

Initialize the Dash app

In [20]:
app = dash.Dash(__name__, 
		# url_base_pathname='/visualisaties/parkeergarages-gent/',
		assets_folder='assets') # Relative path to the folder of css file)
app.title = "Beschikbaarheid fietsen"

# Function to update graph

For color setup, see https://stackoverflow.com/questions/72222646/python-plotly-scattermapbox-define-colors-by-category

In [21]:
# def graph_dott(df):
#     # Replace 'distance' with the actual column name containing the distances in meters
#     df['range_km'] = df['current_range_meters'] / 1000
#     # Create a new column in the DataFrame with formatted distances
#     df['formatted_range'] = df['range_km'].apply(lambda x: f'{x:.1f} km')

#     hover_template = "<b>Dott</b><br>" + \
#                      "Beschikbare afstand: %{customdata[0]}"

#     # Create a map figure using go.Scattermapbox to allow modifying shape of markers
#     fig = go.Figure(go.Scattermapbox(
#         lat=df['lat'],
#         lon=df['lon'],
#         hoverinfo='text',
#         hovertext=df['formatted_range'],
#         mode='markers',
#         marker=dict(
#             size=10,  # Adjust the size as needed
#             symbol='square',
#             color='blue'  # Adjust the color as needed
#         )
#     ))

#     # Update trace 
#     fig.update_traces(
#         hovertemplate=hover_template  # Update hovertemplate
#     )

#     # Set the custom data for hovertemplate
#     fig.update_traces(customdata=df[['formatted_range']].values)

#     # Update the layout to hide the color scale legend
#     fig.update_layout(
#         mapbox=dict(
#             style="carto-positron",
#             zoom=12,
#             center=dict(lon=df['lon'].mean(), lat=df['lat'].mean())
#         ),
#         margin=dict(l=0, r=0, t=0, b=0),
#         coloraxis_showscale=False
#     )

#     return fig


In [22]:
def graph_from_json(json_figures):
    # Create a hovertemplate
    hover_template = "<b>%{customdata[0]</b><br>" + \
                     "Beschikbare afstand: %{customdata[1]}"

    # Create a map figure using go.Scattermapbox to allow modifying shape of markers
    fig = go.Figure()

    for json_figure in json_figures:
        lat = json_figure['lat']
        lon = json_figure['lon']
        formatted_range = f"{json_figure['current_range_meters'] / 1000:.1f} km" if json_figure['current_range_meters'] != None else "N/A"
        brand = json_figure['brand']

        # Add each point to the map
        fig.add_trace(go.Scattermapbox(
            lat=[lat],
            lon=[lon],
            # hoverinfo='text',
            # hovertext=[formatted_range],
            mode='markers',
            marker=go.scattermapbox.Marker(color=color_dict[brand])
            # marker=dict(
            #     size=10,  # Adjust the size as needed
            #     symbol='square',
            #     color='blue'  # Adjust the color as needed
            )
        )

    # Update trace 
    fig.update_traces(
        hovertemplate=hover_template  # Update hovertemplate
    )

    # Set the custom data for hovertemplate
    fig.update_traces(customdata=[[brand, formatted_range] for _ in json_figures])

    # Update the layout to hide the color scale legend
    fig.update_layout(
        mapbox=dict(
            style="carto-positron",
            zoom=12,
            center=dict(lon=json_figures[0]['lon'], lat=json_figures[0]['lat']),
            # color=brand
        ),
        margin=dict(l=0, r=0, t=0, b=0),
        coloraxis_showscale=False
    )

    return fig

# Example usage
# Replace json_figures with your actual list of JSON figures
# fig = graph_dott(json_figures)
# fig.show()


In [23]:
# def graph_dott(df):
#     # Create a hovertemplate
#     # hover_template = "<b>%{hovertext}</b><br>" + \
#     #                  "Fiets ID: %{customdata[0]}<br>" + \
#     #                  "Beschikbare afstand: %{customdata10]}"

#     hover_template = "<b>Dott</b><br>" + \
#                  "Beschikbare afstand: %{customdata[0]}"

#     # Replace 'distance' with the actual column name containing the distances in meters
#     df['range_km'] = df['current_range_meters'] / 1000
#     # Create a new column in the DataFrame with formatted distances
#     df['formatted_range'] = df['range_km'].apply(lambda x: f'{x:.1f} km')

    
#     # Create a map figure using px. Scatter_mapbox
#     fig = px.scatter_mapbox(
#         df,
#         lat=df['lat'],
#         lon=df['lon'],
#         # color=df["vehicle_type_id"],
#         # size="current_range_meters",
#         # hover_name="bike_id",
#         hover_data={"bike_id": True, 
#                     "current_range_meters": True
#                    },
#         # # Create predefined color scale to ensure sufficient contrast, not coming close to white
#         # color_continuous_scale=[
#         #     [0.0, "red"],
#         #     [0.3, "orange"],
#         #     [0.5, "yellow"],
#         #     [0.7, "lime"],
#         #     [1.0, "green"]
#         # ],
#         # color_continuous_scale="RdYlGn",  # Red (low available capacity) to Green (high available capacity)
#         # range_color=[df["is_reserved"].min(), df["is_reserved"].max()],
#         # color_discrete_map={"dott_bicycle": "red", "0": "green"},
#         # For mapbox_styles, see https://plotly.com/python/mapbox-layers/ 
#         mapbox_style="carto-positron", # light
#         # mapbox_style="carto-darkmatter", # dark
#         # mapbox_style="open-street-map", # street style
#         zoom=12,
#         # labels={'availablecapacity': 'Beschikbare parkeerplaatsen'}  # Set the legend label
#     )
    
#     # Update trace 
#     fig.update_traces(
#         # marker_symbol='square', # Update marker symbol to square
#         hovertemplate=hover_template # Update hovertemplate
#     )
       
#     # Set the custom data for hovertemplate
#     fig.update_traces(customdata=df[[
#         # 'bike_id', 
#         'formatted_range'
#     ]].values)
    
    
#     # Update the layout to hide the color scale legend (shows bad on mobile site)
#     fig.update_layout(
#         margin=dict(l=0, r=0, t=0, b=0),
# 		coloraxis_showscale=False,
#         )
    
#     return fig


In [24]:
# dott_graph = graph_dott(df_dott)

In [25]:
json_graph = graph_from_json(data_json_dott_bolt_baqme)

In [26]:
# # *****************************************************************************
# # Show map
# dott_graph.show()
# # *****************************************************************************

# Dash app

## Layout

In [27]:
# Define the app layout, using CSS Bootstrap
app.layout = html.Div([
    html.Link(rel='stylesheet', href='assets/styles.css'),  # Your custom CSS link
    html.Link(rel='stylesheet', href='https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css'),  # Bootstrap CSS link

    html.Div(className='container mt-4', children=[
        html.H1(className='text-center', children="Beschikbaarheid deelfietsen"),

        # html.P(className='text-center', children="Beschikbaarheid van de verschillende parkeergarages binnen het Gentse stadscentrum."),

        # html.Div(className='d-flex justify-content-between align-items-center flex-wrap', children=[ # Make button and update indicator more compact
        #     html.Div(id='last-update-time', className='text-center pt-3 pb-2', children=get_last_update_time('../data/last_update.txt')),
        
        #     html.Div(className='text-center', children=[
        #         html.Button("Update", id="refresh-btn", className='btn btn-primary mt-3 mb-3'),
        #         dcc.Interval(id='refresh-interval-component', interval=10*60*1000, n_intervals=0)
        #     ]),
        # ]),
                        
        # html.Div(className='graph-container custom-graph-container', children=[ # Add custom-graph-container next to standard Bootstrap CSS style
            
        #     # Dropdown for selecting display option
        #     dcc.Dropdown(
        #         id='display-option',
        #         options=[
        #             {'label': 'Parkeergarages', 'value': 'parkings'},
        #             {'label': 'Parkeertariefzones', 'value': 'parking-zones'},
        #             {'label': 'Parkeergarages en parkeertariefzones', 'value': 'parkings_AND_parking-zones'},
        #         ],
        #         value='parkings',  # Set default value
        #         multi=False  # Allow only one option to be selected
        #     ),
        #     # Graph                                                                 
        #     dcc.Graph(id='live-update-graph', figure=update_parkings()),
        #     dcc.Interval(id='update-graph-interval', interval=1*1000, n_intervals=0),
        # ]),        

        html.Div(
            # dcc.Graph(figure=dott_graph)
            dcc.Graph(figure=json_graph)
        ),
        # html.Footer(className='text-center', children=html.P([
        #     "De gegevens zijn beschikbaar via ",
        #     html.A("Stad Gent API", href="https://data.stad.gent/explore/dataset/bezetting-parkeergarages-real-time/table/?sort=-occupation"),
        #     ". De onderliggende code is beschikbaar op ",
        #     html.A("GitHub", href="https://github.com/NT131/parkeergarages_gent"),
            # "."
        # ]))
    ])
])

## Run app

In [28]:
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

# Temp

In [29]:
# # Fetching data function (when using export function)
# def fetch_data_dott_export(url):
#     response = requests.get(url)
#     if response.status_code == 200:
#         data = response.json()  
        
#         # Convert 'last_reported' string to datetime
#         for entry in data:
#             entry['last_reported'] = datetime.utcfromtimestamp(int(entry['last_reported'])).strftime('%Y-%m-%d %H:%M:%S')
        
#         # # Convert to JSON
#         # json_data = json.dumps(data
#         #                        # , indent=2
#         #                       )
        
#         # Write new data to file
#         with open('../data/dott_fetched_data.json', 'w') as json_file:
#             json.dump(data, json_file) 
#         print("Data (Dott bikes) fetched")
#         # return json_data
    
#     else:
#         print("Failed to fetch data (Dott bikes)")

In [30]:
# # Fetching data function (when using API)
# def fetch_data_dott_api(url):
#     response = requests.get(url)
#     if response.status_code == 200:
#         data = response.json()    
#         # Filter out the row with name "Loop"
#         # filtered_data = [record for record in data.get("results", []) if record.get("name") != "The Loop"]
#         data = [record for record in data.get("results", [])]
        
#         # Write new data to file
#         with open('../data/dott_fetched_data.json', 'w') as json_file:
#             json.dump(data, json_file) 
#         print("Data (Dott bikes) fetched")
    
#     else:
#         print("Failed to fetch data (Dott bikes)")

In [31]:
# response = requests.get(url_dott)
# data = response.json() 

In [32]:
# data

In [33]:
# # Convert 'last_reported' string to datetime
# for entry in data:
#     entry['last_reported'] = datetime.utcfromtimestamp(int(entry['last_reported'])).strftime('%Y-%m-%d %H:%M:%S')

# # Create DataFrame
# df = pd.DataFrame(data)
# df

In [34]:
# data_dott_json

In [35]:
	### OPTION 1 ###
	# Fetch data file if it is not yet present
# if not os.path.exists("../data/dott_fetched_data.json"):
#     fetch_data_dott()
	### OPTION 2 ###
	# Fetch data again on each time opening webpage
# fetch_data_dott_api(url_dott)

# data_dott_json = fetch_data_dott_export(url_dott)
# fetch_data_dott_export(url_dott)